## Teste de Dados Geológicos ##

Este notebook faz testes com os dados geológicos fornecidos pelo Humberto. O conjunto de dados está estruturado da seguinte forma: 
- 2259 registros contendo dados de:profundidade, densidade, resistividade média a cada 2.5 ft, resistividade, raios gamma e petecial espontâneo.
- Profundidade, em metros, indo de 68.2 até 519.8 com intervlo de 0.2 m.

In [1]:
## Import all required libraries 
import numpy as np
## Used to find border evidences
import math
from scipy.optimize import dual_annealing

In [2]:
## This function computes the indexes from a list where the condition is true
## call: get_indexes(condicao) - example: get_indexes(x>0)

def get_indexes(self):
    try:
        self = list(iter(self))
    except TypeError as e:
        raise Exception("""'get_indexes' method can only be applied to iterables.{}""".format(str(e)))
    indices = [i for i, x in enumerate(self) if bool(x) == True]
    return(indices)

In [3]:
## O que faz esta função ???

def func_obj_l_L_mu(j, z, n, matdf1, matdf2):
    j = int(np.round(j))
    mue = matdf1[j, 0]
    Le  = matdf1[j, 1]
    mud = matdf2[j, 0]
    Ld  = matdf2[j, 1]
    somaze = sum(z[0: j]) / j
    somalogze = sum(np.log(z[0: j])) / j
    somazd = sum(z[j: n]) / (n - j)
    somalogzd = sum(np.log(z[j: n])) / (n - j)
    #
    aux1 = Le * np.log(Le)
    aux2 = Le * somalogze
    aux3 = Le * np.log(mue)
    aux4 = np.log(math.gamma(Le))
    aux5 = (Le / mue) *  somaze
    #
    aux6  = Ld * np.log(Ld)
    aux7  = Ld * somalogzd
    aux8  = Ld * np.log(mud)
    aux9  = np.log(math.gamma(Ld))
    aux10 = (Ld / mud) * somazd
    a1 =  aux1 + aux2 - aux3 - aux4 - aux5
    a2 =  aux6 + aux7 - aux8 - aux9 - aux10
    #### AAB  Beware! The signal is negative because GenSA finds the point of minimum
    func_obj_l_L_mu = -(j * a1 + (n - j) * a2)
    return func_obj_l_L_mu

In [4]:
## Finds border evidences

def find_evidence(GR, window, overlap):
    print("Computing evidence - this might take a while")
    Le = 4
    Ld = 4
    num_trechos=1+int((len(GR)-window)/(window-overlap))
    print(num_trechos)
    z = np.zeros((num_trechos, window))
    inicio=0
    final=window
    for k in range(num_trechos):
        print(k)
        z[k,:]=GR[inicio:final]
        inicio=final-overlap
        final=inicio+window
    print(z)
    matdf1 =  np.zeros((window-1, 2))
    matdf2 =  np.zeros((window-1, 2))
    evidencias= np.zeros((num_trechos))
    for i in range(num_trechos):
        matdf1[0,0]=z[i,0]
        matdf1[0,1]=Le
        matdf2[0,0]=sum(z[i,1:window]) / (window-1)
        matdf2[0,1]=Ld
        for j in range(1,window-1):
            mue = sum(z[i, 0:j+1]) / (j+1)
            matdf1[j, 0] = mue
            matdf1[j, 1] = Le
            mud = sum(z[i,j+1:window]) / (window - (j+1))
            matdf2[j, 0] = mud
            matdf2[j, 1] = Ld
        lw = [0]
        up = [window - 0]
        ##print(matdf1)
        ##print(matdf2)
        ret = dual_annealing(lambda x:func_obj_l_L_mu(x,z, window-1, matdf1, matdf2), bounds=list(zip(lw, up)), seed=1234)
        evidencias[i] = np.round(ret.x)
    
    return evidencias

In [5]:
GR=[1,1,1,1,2,2,2,2,2,2,2,2,1,1,1,1,1,1,2,2,2,2,1,1,1,1,1,1,2,2,2,2,2,2,1,1,1,1,1,1]
z=GR[0:10]
print(z)

##find_evidence(GR, 10, 4)

[1, 1, 1, 1, 2, 2, 2, 2, 2, 2]


In [6]:
evidencias=find_evidence(GR, 10, 4)
print(evidencias)

Computing evidence - this might take a while
6
0
1
2
3
4
5
[[1. 1. 1. 1. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2. 2. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 2. 2. 2. 2.]
 [2. 2. 2. 2. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 2. 2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 1. 1. 1. 1. 1. 1.]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()